# Initialize libraries

In [1]:
import deeplabcut
from datetime import datetime

from dlchelperclass import DlcHelperClass as dhc

Loading DLC 3.0.0rc5...


# Create new project

In [2]:
# Enter your project name
project_name = 'dlcrnet_stride32_ms5_lr_001_bs_4'
# If you want to use labeled frames from an earlier project,
# make sure to use the same experiment name as in the previous project.
# Enter your experimenter name
experimenter_name = 'conv_vid'
# Specify where you want your project folder to be created
work_dir = '/local/data2/LIA_LIU_PONTUS/LIA_LIU'
# create project path
project_path = work_dir + '/' + project_name + '-' + experimenter_name + '-' + datetime.now().strftime("%Y-%m-%d")
# video path
videos_dir_path = '/local/data2/LIA_LIU/videos_converted'
# Creates a list variable of all videos paths 
video_path = dhc.get_video_paths(path=videos_dir_path)
# Create project
deeplabcut.create_new_project(
    project=project_name,
    experimenter=experimenter_name,
    videos=video_path,
    working_directory=work_dir,
    copy_videos=True
)

# Config yaml path
config_path = project_path + '/config.yaml'

Created "/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/videos"
Created "/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/labeled-data"
Created "/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/training-datasets"
Created "/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/dlc-models"
Copying the videos
/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/videos/GH010346_converted.mp4
/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/videos/240frames_converted.mp4
/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/videos/RF_mapping_untracked_converted.mp4
/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/videos/GH010342_converted.mp4
/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_str

# Initialize paths (For loading existing project)
# skip this cell if you created a new project

In [ ]:
# Enter your project path
project_path = "/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001-conv_vid-2024-11-21"
# Creates path for config file and videos
config_path, video_path = dhc.get_config_and_video_paths(project_path=project_path)

# Extract frames

In [ ]:
# Extract frames for labeling, amount of images and train/test ratio defined in config.yaml
deeplabcut.extract_frames(config_path=config_path, video_path=video_path, mode='automatic')

# Use napari for labeling 

# Creates CSV and h5 files needed for labeling

In [3]:
deeplabcut.create_training_dataset(config=config_path)

/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/labeled-data/GH010346_converted/CollectedData_conv_vid.h5  not found (perhaps not annotated).
/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/labeled-data/GH010345_converted/CollectedData_conv_vid.h5  not found (perhaps not annotated).
/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_lr_001_bs_4-conv_vid-2024-11-21/labeled-data/GH010336_converted/CollectedData_conv_vid.h5  not found (perhaps not annotated).
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.9,
  1,
  (array([382, 487, 351,  14, 145, 422, 753, 253, 175, 231,  31, 235, 764,
          563, 156, 215, 441, 435, 103, 386, 193, 767, 734, 662,  27, 636,
          530, 627, 457, 643, 501, 701,  55, 776, 635, 586, 781,  85, 356,
           40, 122, 362, 666,  75, 476,   8, 693, 239, 365, 150, 285, 604,
          647, 485, 590, 409, 283, 578, 268,  79,   1, 313, 483, 490, 676,
          250, 397, 619, 614, 558, 366, 691, 673, 502, 644, 236, 592, 279,
          251, 620, 315, 399, 517, 567, 724, 363, 113, 413, 252, 350, 638,
           18, 196, 425, 571, 293, 575, 369,  50, 271, 751, 760, 710, 415,
           39, 142, 172, 454, 739,  10, 584,  68, 214, 364, 242, 686, 327,
          471,   2, 775, 625, 230, 202, 473, 447, 580, 258, 261, 774, 374,
          389, 249, 708, 200, 243, 577, 337,  71,  49, 424, 210, 116,  37,
          406,  48, 632, 319, 294, 241, 222, 162,  76,  64, 622,  52, 491,
          597, 432, 519, 564, 773, 402, 750, 680, 118,  12, 157, 628, 263,
          306

# Train network

In [ ]:
deeplabcut.train_network(config_path, maxiters=1_000_000)

# Creates training_stats.csv

In [ ]:
# Needed for plotting losses
deeplabcut.evaluate_network(config_path, plotting=True)

# Makes prediction on videos

In [ ]:
deeplabcut.analyze_videos(config_path, videos=video_path)

# Create new videos with the models predicted labels

In [ ]:
deeplabcut.create_labeled_video(config_path, videos=video_path)

# Evaluate models

In [ ]:
# Creates likelihood.txt file and loss/vall_loss plot
dhc.save_mean_likelihood_to_file(project_path=project_path)
dhc.plot_loss_to_png(project_path)

# Extract outlier frames and re-labeling

In [ ]:
deeplabcut.extract_outlier_frames(config=config_path,
                                   shuffle=1,
                                  automatic=True,
                                  videos=video_path)

In [ ]:
# Open the DeepLabCut GUI to manually correct labels
# Launch the GUI to manually adjust labels on the extracted frames
deeplabcut.refine_labels(project_path)

In [ ]:
deeplabcut.merge_datasets(config=config_path)

In [ ]:
# Create a new training dataset that includes the updated labels
# This prepares the training files with the corrected labels for training
deeplabcut.create_training_dataset(config=config_path)

# Finetune 

In [ ]:
# Make sure you have run the following code line before:
# * deeplabcut.analyze_videos(project_path, video_paths)

In [ ]:
# Retrain the model with the updated dataset
# Use transfer learning to improve the existing model with the new data
deeplabcut.train_network(project_path, shuffle=1, displayiters=100, saveiters=1000)

In [ ]:
# Re-evaluate performance on the updated model
# Analyze the videos again to check if model accuracy has improved after retraining
deeplabcut.analyze_videos(project_path, video_path)

# Predict unseen data

In [ ]:
video_path = "/local/data2/LIA_LIU_PONTUS/LIA_LIU/untrained_videos/dlcrnet_stride32_ms5_test/GH010345_converted.mp4"

In [ ]:
deeplabcut.analyze_videos(config_path, videos=video_path)

In [ ]:
deeplabcut.create_labeled_video(config_path, videos=video_path)

In [ ]:
import os

# Replace with your actual old projects directory
old_project_path = "/local/data2/old_projects/"

# Iterate through each project in the old_project_path
for project in os.listdir(old_project_path):
    project_path = os.path.join(old_project_path, project)  # Construct full project path

    # Check if the path is a directory (to ensure it's a project folder)
    if os.path.isdir(project_path):
        # Call methods from your helper class
        #dhc.save_mean_likelihood_to_file(project_path=project_path)
        dhc.plot_loss_to_png(project_path)
                